In [1]:
import math
import random
from collections import Counter, defaultdict
import heapq

# Get n-grams
def get_ngrams(sentences, n):
    ngrams = []
    for sent in sentences:
        tokens = ["<s>"]*(n-1) + sent.strip().split() + ["</s>"]
        ngrams.extend([tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)])
    return ngrams

In [2]:
# 1. Katz Backoff (Quadrigram)
class KatzBackoff:
    def __init__(self, train, delta=0.5):
        self.delta = delta
        # build counts
        self.unigrams = Counter(get_ngrams(train,1))
        self.bigrams = Counter(get_ngrams(train,2))
        self.trigrams = Counter(get_ngrams(train,3))
        self.quads = Counter(get_ngrams(train,4))
        self.total_unigrams = sum(self.unigrams.values())

    def prob(self, w1,w2,w3,w4):
        # Quadrigram MLE
        quad = (w1,w2,w3,w4)
        tri = (w1,w2,w3)
        if self.quads[quad] > 0 and self.trigrams[tri] > 0:
            return (self.quads[quad] - self.delta) / self.trigrams[tri]

        # Backoff to trigram
        tri2 = (w2,w3,w4)
        bi = (w2,w3)
        if self.trigrams[tri2] > 0 and self.bigrams[bi] > 0:
            return (self.trigrams[tri2] - self.delta) / self.bigrams[bi]

        # Backoff to bigram
        bi2 = (w3,w4)
        if self.bigrams[bi2] > 0 and self.unigrams[(w3,)] > 0:
            return (self.bigrams[bi2] - self.delta) / self.unigrams[(w3,)]

        # Backoff to unigram
        return self.unigrams[(w4,)] / self.total_unigrams

In [3]:
# 2. Kneser-Ney Smoothing (Quadrigram)
class KneserNey:
    def __init__(self, train, discount=0.75):
        self.d = discount
        self.unigrams = Counter(get_ngrams(train,1))
        self.bigrams = Counter(get_ngrams(train,2))
        self.trigrams = Counter(get_ngrams(train,3))
        self.quads = Counter(get_ngrams(train,4))
        self.total_unigrams = sum(self.unigrams.values())

        # continuation counts for lower-order models
        self.continuation_counts = defaultdict(set)
        for (w1,w2,w3,w4) in self.quads:
            self.continuation_counts[(w2,w3,w4)].add(w4)

    def prob_quad(self, w1,w2,w3,w4):
        quad = (w1,w2,w3,w4)
        tri = (w1,w2,w3)

        # numerator with discount
        numerator = max(self.quads[quad] - self.d, 0) / self.trigrams[tri] if self.trigrams[tri] > 0 else 0

        # backoff weight
        lambda_factor = (self.d * len([q for q in self.quads if q[:3]==tri])) / self.trigrams[tri] if self.trigrams[tri] > 0 else 1

        # continuation probability (using trigram model)
        cont_prob = self.prob_tri(w2,w3,w4)
        return numerator + lambda_factor * cont_prob

    def prob_tri(self, w2,w3,w4):
        tri = (w2,w3,w4)
        bi = (w2,w3)
        numerator = max(self.trigrams[tri] - self.d, 0) / self.bigrams[bi] if self.bigrams[bi] > 0 else 0
        lambda_factor = (self.d * len([t for t in self.trigrams if t[:2]==bi])) / self.bigrams[bi] if self.bigrams[bi] > 0 else 1
        return numerator + lambda_factor * self.prob_bi(w3,w4)

    def prob_bi(self, w3,w4):
        bi = (w3,w4)
        numerator = max(self.bigrams[bi] - self.d, 0) / self.unigrams[(w3,)] if self.unigrams[(w3,)] > 0 else 0
        lambda_factor = (self.d * len([b for b in self.bigrams if b[0]==w3])) / self.unigrams[(w3,)] if self.unigrams[(w3,)] > 0 else 1
        return numerator + lambda_factor * self.prob_uni(w4)

    def prob_uni(self, w):
        return self.unigrams[(w,)] / self.total_unigrams

In [4]:
# 3. Sentence Generation

# (a) Greedy Generation
def generate_greedy(model, max_len=15):
    sentence = ["<s>", "<s>", "<s>"]
    for _ in range(max_len):
        candidates = {}
        for word in model.unigrams.keys():
            prob = model.prob(sentence[-3], sentence[-2], sentence[-1], word[0]) if isinstance(model, KatzBackoff) else model.prob_quad(sentence[-3], sentence[-2], sentence[-1], word[0])
            candidates[word[0]] = prob
        next_word = max(candidates, key=candidates.get)  # pick word with highest prob
        if next_word == "</s>":
            break
        sentence.append(next_word)
    return " ".join(sentence[3:])  # remove <s>

In [5]:
# (b) Beam Search
def generate_beam(model, beam_size=20, max_len=15):
    beam = [(["<s>", "<s>", "<s>"], 0.0)]  # (sequence, log-probability)

    for _ in range(max_len):
        new_beam = []
        for seq, score in beam:
            for word in model.unigrams.keys():
                prob = model.prob(seq[-3], seq[-2], seq[-1], word[0]) if isinstance(model, KatzBackoff) else model.prob_quad(seq[-3], seq[-2], seq[-1], word[0])
                new_seq = seq + [word[0]]
                new_score = score + math.log(prob + 1e-12)
                new_beam.append((new_seq, new_score))
        # keep top beam_size
        beam = heapq.nlargest(beam_size, new_beam, key=lambda x: x[1])
        # stop if all sequences ended
        if all(seq[-1] == "</s>" for seq,_ in beam):
            break

    best_seq = max(beam, key=lambda x: x[1])[0]
    return " ".join(best_seq[3:-1])  # remove <s> and </s>

In [ ]:
# Example
if __name__ == "__main__":
    # load your corpus
    with open("tokenized_bengali.txt", "r", encoding="utf-8") as f:
        sentences = f.readlines()

    # train models
    katz = KatzBackoff(sentences)
    kn = KneserNey(sentences)

    # Generate sentences
    print("Greedy (Katz):", generate_greedy(katz))
    print("Beam Search (Katz):", generate_beam(katz))

    print("Greedy (Kneser-Ney):", generate_greedy(kn))
    print("Beam Search (Kneser-Ney):", generate_beam(kn))


Greedy (Katz): 
Beam Search (Katz): e d d a n e s h S c i e n c
Greedy (Kneser-Ney): 
